In [1]:
import numpy as np

def E(u,v):
    return np.exp(u)+np.exp(2*v)+np.exp(u*v)+u*u+2*v*v-2*u*v-3*u-2*v

def gradU(func,u,v):
    return (func(u+0.0001, v)-func(u-0.0001, v)) / 0.0002

def gradV(func,u,v):
    return (func(u, v+0.0001)-func(u, v-0.0001)) / 0.0002

def update(N, rate):
    u,v = 0,0
    for i in range(N):
        gdu = gradU(E,u,v)
        gdv = gradV(E,u,v)
        u = u - rate*gdu
        v = v - rate*gdv
    return u,v

def six(u,v):
    gdu = gradU(E,u,v)
    gdv = gradV(E,u,v)
    return gdu,gdv

def eight(u,v):
    gu = gradU(E,u,v)
    gv = gradV(E,u,v)
    guu = gradU(lambda u,v:gradU(E,u,v), u,v)
    gvv = gradV(lambda u,v:gradV(E,u,v), u,v)
    guv = gradV(lambda u,v:gradU(E,u,v), u,v)
    gvu = gradU(lambda u,v:gradV(E,u,v), u,v)
    return (guu/2, gvv/2, guv, gu, gv, E(u,v))


def newton(N):
    u,v = 0,0
    for i in range(N):
        gu = gradU(E,u,v)
        gv = gradV(E,u,v)
        guu = gradU(lambda u,v:gradU(E,u,v),u,v)
        gvv = gradV(lambda u,v:gradV(E,u,v),u,v)
        guv = gradU(lambda u,v:gradV(E,u,v),u,v)
        gvu = gradV(lambda u,v:gradU(E,u,v),u,v)
        hesse = np.mat([[guu, guv], [gvu, gvv]])
        grad = np.array([[gu],[gv]])
        delta = hesse.I*grad
        delta = delta.tolist()
        u = u - delta[0][0]
        v = v - delta[1][0]
    return u,v

In [5]:
# 13
import numpy as np
import random

def sign(x):
    if x>0:
        return 1
    else:
        return -1

def target(x1,x2):
    return np.sign(x1**2+x2**2-0.6)

def genData(N):
    xs = []
    ys = []
    for i in range(N):
        x1 = np.random.uniform(-1,1)
        x2 = np.random.uniform(-1,1)
        y = target(x1,x2)
        prob = np.random.uniform(0,1)
        if prob<0.1:
            y = -y
        xs.append([1,x1,x2])
        ys.append([y])
    return np.mat(xs), np.array(ys)

def LinearReg(x,y):
    p_inv = np.linalg.pinv(x)
    w = p_inv*y
    return np.array(w)

def ErrorRate(w,x,y):
    yhat = np.array(x*w)
    yhat = list(map(sign,yhat))
    ytrue = y.reshape(-1).tolist()    
    errorNum = np.sum(np.array(yhat) != np.array(ytrue))
    return errorNum / len(y)

def main():
    N = 1000
    errorSum = 0
    for i in range(N):
        x,y = genData(1000)
        w = LinearReg(x,y)
        errorSum += ErrorRate(w,x,y)
    print('error in sample is ',errorSum/N)


In [2]:
# 14
import numpy as np
import random

def sign(x):
    if x>0:
        return 1
    else:
        return -1

def target(x1,x2):
    return np.sign(x1**2+x2**2-0.6)

def gendata(N):
    xs = []
    ys = []
    for i in range(N):
        x1 = np.random.uniform(-1,1)
        x2 = np.random.uniform(-1,1)
        y = target(x1,x2)
        prob = np.random.uniform(0,1)
        if prob<0.1:
            y = -y
        xs.append([1,x1,x2,x1*x2,x1*x1,x2*x2])
        ys.append([y])
    return np.mat(xs), np.array(ys)

def LinearReg(x,y):
    p_inv = np.linalg.pinv(x)
    w = p_inv*y
    return np.array(w)

def ErrorRate(w,x,y):
    yhat = np.array(x*w)
    yhat = list(map(sign,yhat))
    ytrue = y.reshape(-1).tolist()    
    errorNum = np.sum(np.array(yhat) != np.array(ytrue))
    return errorNum / len(y)

def main14():
    N = 1000
    W = []
    errorSum = 0
    for i in range(N):
        x,y = gendata(1000)
        w = LinearReg(x,y)
        errorSum += ErrorRate(w,x,y)
    print('error in sample is ',errorSum/N)

def mainn():
    N = 1000
    W = np.array([])
    errorSum = 0
    for i in range(N):
        x,y = gendata(1000)
        w = LinearReg(x,y)
        W = np.append(W,w)
        w2 = W[1::6]
    return w2

In [56]:
# 18 - 20

# add intercept
def preprocess(x):
    n = x.shape[0]
    return np.c_[np.ones(n),x]

def getdata():
    t_data = np.genfromtxt('train.txt')
    x = t_data[:,:-1]
    train_x = preprocess(x)
    train_y = t_data[:,-1].reshape(-1,1)
    
    d_data = np.genfromtxt('test.txt')
    x1 = d_data[:,:-1]
    test_x = preprocess(x1)
    test_y = d_data[:,-1].reshape(-1,1)
    return train_x,train_y,test_x,test_y

def sigmoid(s):
    h = 1 / (1+np.exp(-s))
    return h
    
def hypo(x,w):
    return sigmoid(w*x)
    
def gd(x,y,w):
    temp1 = -x.dot(w)*y
    temp2 = sigmoid(temp1)
    temp3 = -x*y
    grad = np.mean(temp2*temp3, axis=0).reshape(-1,1)
    return grad

def initial_w(x):
    n,m = x.shape
    return np.zeros((m,1))
    
def main18(learning_rate):
    train_x,train_y,test_x,test_y = getdata()
    w = initial_w(train_x)
    N = 2000
    for i in range(N):
        grad = gd(train_x,train_y,w)
        w = w - learning_rate*grad    
    y_pred = np.sign(test_x.dot(w))
    errorRate = np.sum(y_pred != test_y) / test_y.shape[0]
    return errorRate

def main20(learning_rate):
    train_x,train_y,test_x,test_y = getdata()
    n,m = train_x.shape
    w = np.zeros((21,1))
    j = 0
    for i in range(2000):
        x = train_x[j,:].reshape(1,-1)
        y = train_y[j]
        grad = gd(x,y,w)
        w = w - learning_rate*grad
        j += 1
        j = j%n
    y_pred = np.sign(test_x.dot(w))
    errorRate = np.sum(y_pred != test_y) / test_y.shape[0]
    return errorRate